<a href="https://colab.research.google.com/github/vinhpeters/tf-exam/blob/main/03_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Computer Vision with Covolutional Neural Networks

In [ ]:
import tensorflow as tf
import zipfile

## Using subset of Food 101 dataset

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

--2023-04-06 14:45:21--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 142.250.153.128, 142.250.145.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109540975 (104M) [application/zip]
Saving to: ‘pizza_steak.zip’

pizza_steak.zip     100%[===================>] 104.47M  33.5MB/s    in 3.1s    

2023-04-06 14:45:25 (33.5 MB/s) - ‘pizza_steak.zip’ saved [109540975/109540975]



In [ ]:
zip_ref = zipfile.ZipFile('pizza_steak.zip')
zip_ref.extractall()
zip_ref.close()

## Data exploration

In [ ]:
!ls pizza_steak/train/pizza

In [ ]:
import os 

for dirpath, dirnames, filenames in os.walk('pizza_steak'):
  print(f'Dirs: {len(dirnames)}, imgs:{len(filenames)} in {dirpath}')

In [ ]:
num_steak=len(os.listdir('pizza_steak/train/steak'))
num_steak

In [ ]:
num_pizza=len(os.listdir('pizza_steak/train/pizza'))
num_pizza

## Get class names programmitically

In [ ]:
import pathlib
import numpy as np 
data_dir = pathlib.Path('pizza_steak/train')
class_names = np.array(sorted(item.name for item in data_dir.glob('*')))

In [ ]:
class_names

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  target_folder = target_dir+target_class

  random_image = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_image[0] )
  plt.imshow(img)
  plt.title(target_class)
  plt.axis('off');

  print(f'Image shape: {img.shape}')
  return img

In [ ]:
img = view_random_image(target_dir='pizza_steak/train/', target_class='steak')

## An end-to-end example

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_dir = 'pizza_steak/train'
test_dir = 'pizza_steak/test'


train_data = train_datagen.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224,224), class_mode ='binary', seed=42)
valid_data = train_datagen.flow_from_directory(directory=test_dir, batch_size=32, target_size=(224,224), class_mode ='binary', seed=42)


In [ ]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters =10, kernel_size=3, activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding='valid'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model_1.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

histor_1=model_1.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data, validation_steps=len(valid_data))


Epoch 1/5
47/47 [==============================] - 89s 2s/step - loss: 0.5636 - accuracy: 0.6960 - val_loss: 0.4527 - val_accuracy: 0.7720
Epoch 2/5
47/47 [==============================] - 86s 2s/step - loss: 0.4156 - accuracy: 0.8167 - val_loss: 0.3399 - val_accuracy: 0.8500
Epoch 3/5
47/47 [==============================] - 86s 2s/step - loss: 0.4084 - accuracy: 0.8180 - val_loss: 0.3470 - val_accuracy: 0.8540
Epoch 4/5
47/47 [==============================] - 85s 2s/step - loss: 0.3596 - accuracy: 0.8480 - val_loss: 0.3212 - val_accuracy: 0.8720


KeyboardInterrupt: ignored

In [ ]:
model_1.evaluate(valid_data)

In [ ]:
tf.random.set_seed(42)

model_2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_2 = model_2.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data, validation_steps=len(valid_data))


In [ ]:
model_2.summary()

In [ ]:
tf.random.set_seed(42)

model_3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_3.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_3 = model_3.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data, validation_steps=len(valid_data))


In [ ]:
model_3.summary()

In [ ]:
plt.figure()
plt.subplot(1,2,1)
steak_img = view_random_image('pizza_steak/train/', 'steak') 
plt.subplot(1,2,2)
pizza_img = view_random_image('pizza_steak/train/', 'pizza')

In [ ]:
train_dir = 'pizza_steak/train'
test_dir = 'pizza_steak/test'

## Batching

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224,224), class_mode ='binary', seed=42)
test_data = train_datagen.flow_from_directory(directory=test_dir, batch_size=32, target_size=(224,224), class_mode ='binary', seed=42)

In [ ]:
images, labels = train_data.next()
len(images), len(labels)

In [ ]:
images[:2], images[0].shape

In [ ]:
labels

In [ ]:
## Baseline CNN Model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

In [ ]:
model_4 = Sequential([
    Conv2D(filters=10, kernel_size=3,strides=1,padding='valid',input_shape=(224,224,3),activation='relu'),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    Flatten(),
    Dense(1, activation='sigmoid')
])


model_4.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_4 = model_4.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data, validation_steps=len(test_data))

In [ ]:
import pandas as pd

pd.DataFrame(history_4.history).plot(figsize=(10,7));

In [ ]:
def plot_loss_curves(history):
  '''
  Returns seperate loss curves for training and validation metrics
  '''

  loss = history.history['loss']
  accuracy = history.history['accuracy']
  val_loss = history.history['val_loss']
  val_accuracy = history.history['val_accuracy']

  epochs=range(len(history.history['loss']))

  plt.plot(epochs,loss, label='training_loss')
  plt.plot(epochs,val_loss, label='val_loss')
  plt.title('loss')
  plt.xlabel('epochs')
  plt.legend()

  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs,val_accuracy, label='val_accuracy')
  plt.title('accuracy')
  plt.xlabel('epochs')
  plt.legend()


In [ ]:
plot_loss_curves(history_4)

In [ ]:
model_5 = Sequential([
    Conv2D(filters=10, kernel_size=3,strides=1,padding='valid',input_shape=(224,224,3),activation='relu'),
    MaxPool2D(pool_size=2),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(1, activation='sigmoid')
])


model_5.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_5 = model_5.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data, validation_steps=len(test_data))

In [ ]:
model_5.summary()

In [ ]:
plot_loss_curves(history_5)

## Data augmentation

In [ ]:
train_datagen_augmented = ImageDataGenerator(rescale=1./255, rotation_range=0.2, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data_augmented = train_datagen_augmented.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224,224), class_mode ='binary', shuffle=False)
train_data= train_datagen.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224,224), class_mode ='binary', shuffle=False)

test_data = train_datagen.flow_from_directory(directory=test_dir, batch_size=32, target_size=(224,224), class_mode ='binary', seed=42)

In [ ]:
images, labels=train_data.next()
augmented_images, augmented_labels = train_data_augmented.next()

In [ ]:
import random

random_number = random.randint(0,31)
plt.imshow(images[random_number])
plt.title('Original Image')
plt.axis(False)
plt.figure();
plt.imshow(augmented_images[random_number])
plt.title('Augmented Image')
plt.axis(False);




In [ ]:
model_6 = Sequential([
    Conv2D(filters=10, kernel_size=3,strides=1,padding='valid',input_shape=(224,224,3),activation='relu'),
    MaxPool2D(pool_size=2),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(1, activation='sigmoid')
])


model_6.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_6 = model_6.fit(train_data_augmented, epochs=5, steps_per_epoch=len(train_data_augmented), validation_data=valid_data, validation_steps=len(test_data))

In [ ]:
train_data_augmented = train_datagen_augmented.flow_from_directory(directory=train_dir, batch_size=32, target_size=(224,224), class_mode ='binary')

In [ ]:
model_7 = Sequential([
    Conv2D(filters=10, kernel_size=3,strides=1,padding='valid',input_shape=(224,224,3),activation='relu'),
    MaxPool2D(pool_size=2),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(1, activation='sigmoid')
])


model_7.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_7 = model_7.fit(train_data_augmented, epochs=5, steps_per_epoch=len(train_data_augmented), validation_data=valid_data, validation_steps=len(test_data))

In [ ]:
plot_loss_curves(history_7)

In [ ]:
model_8 = Sequential([
    Conv2D(filters=10, kernel_size=3,strides=1,padding='valid',input_shape=(224,224,3),activation='relu'),
    MaxPool2D(pool_size=2),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Conv2D(filters=10, kernel_size=3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(1, activation='sigmoid')
])


model_8.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
history_8 = model_8.fit(train_data_augmented, epochs=5, steps_per_epoch=len(train_data_augmented), validation_data=valid_data, validation_steps=len(test_data))